In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline, RectBivariateSpline

# Astronomy
import astropy
from astropy.units import au, minute

from typing import List, Optional

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_pos
from asteroid_spline import get_df_shape, make_spline_df, spline_ast_data
from planets_interp import get_earth_pos
from asteroid_direction import calc_dir_linear, calc_dir_linear_topos, light_time_iter, calc_dir_spline
from asteroid_direction_test import jpl_ast_dir_populate
from ra_dec import radec2dir
from ra_dec_test import direction_diff
from db_utils import sp2df, df2db

# Test asteroid direction with iterated splines

In [3]:
# Import the JPL asteroid directions
df = sp2df(sp_name='JPL.AstrometricDirectionTest', params={'ObservatoryShortname':'Geocenter'})

In [4]:
# The Horizons DataFrame
df

,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tObs,ux,uy,uz,delta
0,1,69120000,48000.0,-0.880028,-0.483807,-0.000050,-1.155150,2.297484,0.282873,-0.009457,-0.005464,0.001583,23.363621,48000.0,-0.097885,0.990090,0.100704,2.809226
1,1,69127200,48005.0,-0.836785,-0.557668,-0.000054,-1.202187,2.269674,0.290725,-0.009357,-0.005659,0.001558,23.833511,48005.0,-0.127458,0.986641,0.101460,2.865725
2,1,69134400,48010.0,-0.787422,-0.627462,-0.000056,-1.248709,2.240893,0.298453,-0.009251,-0.005853,0.001533,24.289747,48010.0,-0.157895,0.982153,0.102200,2.920583
3,1,69141600,48015.0,-0.732357,-0.692671,-0.000054,-1.294694,2.211151,0.306052,-0.009142,-0.006044,0.001507,24.731042,48015.0,-0.189059,0.976556,0.102932,2.973644
4,1,69148800,48020.0,-0.672042,-0.752868,-0.000054,-1.340122,2.180457,0.313519,-0.009028,-0.006233,0.001480,25.156619,48020.0,-0.220819,0.969791,0.103659,3.024815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48011,16,90691200,62980.0,-0.842921,-0.549267,0.000119,-0.813070,2.724700,-0.105745,-0.010217,-0.001563,0.000351,27.244467,62980.0,0.009167,0.999436,-0.032318,3.275856
48012,16,90698400,62985.0,-0.794209,-0.619483,0.000126,-0.864021,2.716450,-0.103974,-0.010163,-0.001737,0.000357,27.764006,62985.0,-0.020859,0.999296,-0.031185,3.338325
48013,16,90705600,62990.0,-0.739772,-0.685171,0.000131,-0.914696,2.707332,-0.102170,-0.010107,-0.001910,0.000364,28.265182,62990.0,-0.051416,0.998224,-0.030103,3.398587
48014,16,90712800,62995.0,-0.680044,-0.745908,0.000132,-0.965081,2.697356,-0.100334,-0.010047,-0.002080,0.000370,28.747137,62995.0,-0.082410,0.996174,-0.029068,3.456536


In [5]:
# Column names for astrometric and apparent direction
cols_u_ast = ['ux_ast', 'uy_ast', 'uz_ast']
cols_u_app = ['ux_app', 'uy_app', 'uz_app']

# Column names for testing 
cols_q_obs = ['qObs_x', 'qObs_y', 'qObs_z']
cols_q_ast = ['qAst_x', 'qAst_y', 'qAst_z']
cols_v_ast = ['vAst_x', 'vAst_y', 'vAst_z']
cols_u = ['ux', 'uy', 'uz']

In [6]:
# Extract arrays from Horizons DataFrame
asteroid_id = df.AsteroidID.values
t_ast = df.tAst.values
q_obs = df[cols_q_obs].values
t_obs = df.tObs.values
light_time = df.LightTime.values
u_jpl = df[cols_u].values

In [7]:
# Get asteroid orbital elements
from asteroid_element import get_ast_elts_ts

In [8]:
n0=0
n1=20
mjd0=48000
mjd1=63000
elt = get_ast_elts_ts(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [9]:
elt

,TimeID,AsteroidID,mjd,a,e,inc,Omega,omega,f,M,WindingNumber
0,69120000,1,48000.0,2.768788,0.077540,0.185062,1.408422,1.237946,-0.601468,-0.517779,0
1,69125760,1,48004.0,2.768752,0.077525,0.185061,1.408417,1.237928,-0.584374,-0.502840,0
2,69131520,1,48008.0,2.768714,0.077510,0.185060,1.408411,1.237912,-0.567257,-0.487900,0
3,69137280,1,48012.0,2.768677,0.077495,0.185059,1.408406,1.237895,-0.550118,-0.472959,0
4,69143040,1,48016.0,2.768639,0.077479,0.185057,1.408400,1.237879,-0.532956,-0.458018,0
...,...,...,...,...,...,...,...,...,...,...,...
71264,90696960,19,62984.0,2.443794,0.156396,0.027463,-2.601255,-3.097766,73.189299,67.175352,11
71265,90702720,19,62988.0,2.443777,0.156391,0.027463,-2.601266,-3.097692,73.204641,67.193281,11
71266,90708480,19,62992.0,2.443760,0.156386,0.027463,-2.601276,-3.097618,73.220050,67.211209,11
71267,90714240,19,62996.0,2.443741,0.156381,0.027463,-2.601287,-3.097543,73.235525,67.229138,11


In [13]:
from topos import calc_topos

In [ ]:
calc_topos(obstime_mjd=)

In [10]:
from asteroid_spline import make_spline_ast_elt, make_spline_ast_pos

In [11]:
n0=1
n1=17
mjd0=57000
mjd1=61000

site_name = 'geocenter'
obs_name = 'Geocenter'

In [12]:
# Import the JPL asteroid directions
sp_name = 'JPL.AstrometricDirectionTest'
df_jpl = sp2df(sp_name=sp_name)

# Mask data down to the selected date range
mask =  (mjd0 <= df_jpl.tObs) & (df_jpl.tObs <= mjd1)
df_jpl = df_jpl[mask].reset_index(drop=True)

sp2df(sp_name) failed!
Traceback (most recent call last):
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1628, in _execute_context
    self.dialect.do_execute(
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 663, in do_execute
    cursor.execute(statement, parameters)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/lib/python3.8/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/home/michael/Anaconda/anaconda3-2020.11/envs/kepler/l

AttributeError: 'NoneType' object has no attribute 'tObs'

In [13]:
df_jpl

,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tObs,ux,uy,uz,delta
0,1,82080000,57000.0,0.230829,0.957230,-0.000164,-0.608331,-2.738365,0.026526,0.009594,-0.002934,-0.001861,31.518399,57000.0,-0.221479,-0.975140,0.007053,3.789751
1,1,82087200,57005.0,0.145017,0.973138,-0.000166,-0.560260,-2.752576,0.017219,0.009634,-0.002750,-0.001862,31.536274,57005.0,-0.186046,-0.982530,0.004595,3.791900
2,1,82094400,57010.0,0.058068,0.981505,-0.000170,-0.512000,-2.765867,0.007906,0.009670,-0.002566,-0.001863,31.524375,57010.0,-0.150445,-0.988616,0.002141,3.790470
3,1,82101600,57015.0,-0.029333,0.982221,-0.000171,-0.463569,-2.778239,-0.001409,0.009702,-0.002382,-0.001863,31.482382,57015.0,-0.114763,-0.993393,-0.000317,3.785420
4,1,82108800,57020.0,-0.116466,0.975265,-0.000169,-0.414984,-2.789692,-0.010724,0.009731,-0.002198,-0.001863,31.410313,57020.0,-0.079092,-0.996863,-0.002784,3.776755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12811,16,87811200,60980.0,0.773691,0.611912,0.000099,1.021725,2.413031,-0.140937,-0.009801,0.005248,0.000019,15.165857,60980.0,0.136075,0.987675,-0.077343,1.823532
12812,16,87818400,60985.0,0.715961,0.676486,0.000092,0.972517,2.438778,-0.140813,-0.009882,0.005051,0.000031,14.856968,60985.0,0.143675,0.986477,-0.078877,1.786392
12813,16,87825600,60990.0,0.652798,0.735916,0.000089,0.922915,2.463538,-0.140631,-0.009958,0.004853,0.000042,14.589457,60990.0,0.154039,0.984803,-0.080218,1.754226
12814,16,87832800,60995.0,0.584618,0.789732,0.000088,0.872940,2.487307,-0.140394,-0.010031,0.004654,0.000053,14.367795,60995.0,0.166954,0.982606,-0.081318,1.727574


In [14]:
u_jpl = df_jpl[cols_u_ast].values

In [15]:
# Spline for asteroid position
spline_q_ast = make_spline_ast_elt(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [16]:
from asteroid_direction import calc_dir_ast2obs
ts=df_jpl.tObs.values
asteroid_id=df_jpl.AsteroidID.values
site_name='geocenter'
iters=2

In [17]:
df_mse = calc_dir_ast2obs(ts=ts, asteroid_id=asteroid_id, site_name=site_name, iters=iters)

In [18]:
df_mse

,AsteroidID,tObs,LightTime,ux,uy,uz
0,1,57000.0,31.518524,-0.221485,-0.975138,0.007054
1,1,57005.0,31.536438,-0.186052,-0.982529,0.004596
2,1,57010.0,31.524576,-0.150452,-0.988615,0.002141
3,1,57015.0,31.482620,-0.114770,-0.993392,-0.000316
4,1,57020.0,31.410586,-0.079098,-0.996863,-0.002784
...,...,...,...,...,...,...
12811,16,60980.0,28.934965,-0.984901,0.123093,-0.121732
12812,16,60985.0,28.565000,-0.987638,0.093795,-0.125590
12813,16,60990.0,28.171823,-0.989418,0.065208,-0.129618
12814,16,60995.0,27.756251,-0.990297,0.037434,-0.133829


In [19]:
u_mse = df_mse[cols_u].values

In [20]:
du = (u_mse - u_jpl)

In [21]:
err = np.sqrt(np.sum(np.square(du), axis=-1))

In [22]:
err

array([6.48893568e-06, 6.47283577e-06, 6.45092330e-06, ...,
       1.46819265e+00, 1.49510392e+00, 1.52223144e+00])

In [23]:
idx = np.argmax(err)
df_mse[]idx-4:idx+4

,AsteroidID,tObs,LightTime,ux,uy,uz
12731,16,60580.0,14.868227,0.086758,0.974707,0.205961
12732,16,60585.0,14.446095,0.066772,0.975807,0.208188
12733,16,60590.0,14.030987,0.049436,0.976388,0.210292
12734,16,60595.0,13.625637,0.034964,0.976596,0.212221
12735,16,60600.0,13.232855,0.023547,0.976569,0.213914
12736,16,60605.0,12.855446,0.015374,0.976427,0.215298
12737,16,60610.0,12.496690,0.010640,0.976272,0.216287
12738,16,60615.0,12.160592,0.009500,0.976175,0.216777


In [24]:
df_jpl[idx-4:idx+4]

,AsteroidID,ObservatoryID,TimeID,tAst,RA_ast,DEC_ast,ux_ast,uy_ast,uz_ast,RA_app,DEC_app,ux_app,uy_app,uz_app,LightTime,tObs
12731,16,0,87235200,60579.988616,309.863994,-18.092383,0.609276,-0.792940,0.005301,310.218393,-18.005147,0.614082,-0.789225,0.005268,16.392571,60580.0
12732,16,0,87242400,60584.988319,310.089947,-18.130739,0.612015,-0.790838,0.003696,310.443987,-18.043203,0.616803,-0.787109,0.003662,16.819956,60585.0
12733,16,0,87249600,60589.988007,310.477089,-18.123530,0.616939,-0.787008,0.002166,310.830568,-18.035321,0.621699,-0.783253,0.002133,17.269292,60590.0
12734,16,0,87256800,60594.987683,311.019369,-18.071135,0.623940,-0.781472,0.000713,311.372340,-17.981953,0.628665,-0.777676,0.000680,17.736594,60595.0
12735,16,0,87264000,60599.987348,311.708411,-17.974366,0.632868,-0.774260,-0.000664,312.060510,-17.884030,0.637545,-0.770413,-0.000696,18.218164,60600.0
12736,16,0,87271200,60604.987006,312.534813,-17.834239,0.643552,-0.765400,-0.001968,312.886036,-17.742389,0.648173,-0.761490,-0.002000,18.710905,60605.0
12737,16,0,87278400,60609.986658,313.489997,-17.651404,0.655825,-0.754906,-0.003205,313.840210,-17.557918,0.660381,-0.750924,-0.003237,19.212110,60610.0
12738,16,0,87285600,60614.986306,314.565697,-17.426250,0.669520,-0.742781,-0.004378,314.914568,-17.330951,0.673997,-0.738721,-0.004408,19.718958,60615.0
